# __Simple Models__

# Carga de Datos

In [1]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.sparse.linalg import svds
from sklearn.neighbors import NearestNeighbors

In [2]:
# Cargar los datos
clients = pd.read_csv('atributos.csv')
transactions = pd.read_csv('transacciones.csv')

# Transformacion de Datos

In [3]:
# Renombrar columnas
new_cols = ['UNNAMED: 0', 'ACCOUNT_ID', 'BUSINESS_SEGMENT', 'TOTAL_VOLUME', 'DIFF_SKU_BY_ORDER',
            'TOTAL_DIFF_SKU', 'CONCENTRATION', 'NSE', 'UNIQUE_SEGMENT', 'CHANNEL']

clients.columns = new_cols

In [4]:
# Eliminar la columna 'UNNAMED: 0'
clients.drop('UNNAMED: 0', axis=1, inplace=True)

# Eliminar la columna 'Unnamed: 0'
transactions.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
# Convertir INVOICE_DATE a datetime
transactions['INVOICE_DATE'] = pd.to_datetime(transactions['INVOICE_DATE'], format='%Y%m%d')

In [6]:
# Unir los dataframes
mdf = pd.merge(transactions, clients, on='ACCOUNT_ID', how='left')

# Feature Engineering

In [7]:
# Crear columnas 'MES', 'DIA', 'DAY_OF_WEEK'
mdf['MES'] = mdf['INVOICE_DATE'].dt.month
mdf['DIA'] = mdf['INVOICE_DATE'].dt.day
mdf['DAY_OF_WEEK'] = mdf['INVOICE_DATE'].dt.dayofweek

In [8]:
# Crear características de desfase
mdf['LAG_1'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(1)
mdf['LAG_2'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(2)
mdf['LAG_3'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(3)
mdf['LAG_4'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(4)
mdf['LAG_5'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(5)
mdf['LAG_6'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(6)
mdf['LAG_7'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(7)
mdf['LAG_8'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(8)
mdf['LAG_9'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(9)
mdf['LAG_10'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(10)
mdf['LAG_11'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(11)
mdf['LAG_12'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(12)
mdf['LAG_13'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(13)
mdf['LAG_14'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(14)

# Crear medias moviles
mdf['MA_3'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].transform(lambda x: x.rolling(3).mean())
mdf['MA_6'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].transform(lambda x: x.rolling(6).mean())
mdf['MA_9'] = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].transform(lambda x: x.rolling(9).mean())

In [9]:
# Calcular la fecha máxima de transacciones
last_date = mdf['INVOICE_DATE'].max()

# Calcular recencia
recency_df = mdf.groupby('ACCOUNT_ID').agg(last_purchase=('INVOICE_DATE', 'max')).reset_index()
recency_df['RECENCIA'] = (last_date - recency_df['last_purchase']).dt.days

# Calcular la frecuencia
frequency_df = mdf.groupby('ACCOUNT_ID')['ORDER_ID'].nunique().reset_index()
frequency_df.columns = ['ACCOUNT_ID', 'FRECUENCIA']

# Calcular el Volumen
monetary_df = mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].sum().reset_index()
monetary_df.columns = ['ACCOUNT_ID', 'TOTAL_ITEMS']

rfm_df = pd.merge(recency_df[['ACCOUNT_ID', 'RECENCIA']],
                  frequency_df[['ACCOUNT_ID', 'FRECUENCIA']],
                  on='ACCOUNT_ID')
rfm_df = pd.merge(rfm_df, monetary_df[['ACCOUNT_ID', 'TOTAL_ITEMS']], on='ACCOUNT_ID')

rfm_df

,ACCOUNT_ID,RECENCIA,FRECUENCIA,TOTAL_ITEMS
0,22204,50,3,5.0
1,22247,1,6,25.0
2,22456,11,9,123.0
3,22477,0,16,86.0
4,22639,5,9,85.0
...,...,...,...,...
4530,676899,8,8,70.0
4531,676938,0,17,294.0
4532,677161,5,4,53.0
4533,677195,8,13,486.0


In [10]:
# Fusionar las tablas
df_merged = pd.merge(mdf, rfm_df, how='left', on='ACCOUNT_ID')

# Verificar el DataFrame resultante
df_merged.head()

,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ORDER_ID,ITEMS_PHYS_CASES,BUSINESS_SEGMENT,TOTAL_VOLUME,DIFF_SKU_BY_ORDER,TOTAL_DIFF_SKU,CONCENTRATION,...,LAG_11,LAG_12,LAG_13,LAG_14,MA_3,MA_6,MA_9,RECENCIA,FRECUENCIA,TOTAL_ITEMS
0,430606,7038,2022-07-29,512-3880249-0,100.0,MediumUsage,326.49782,3.0000,34.0,Alto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,37,2520.0
1,323267,14933,2022-07-29,512-3882307-0,1.0,MinimalUsage,1.18812,5.0000,12.0,Medio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,9,54.0
2,357825,21971,2022-07-23,512-3852880-0,8.0,MediumUsage,112.64566,14.5455,56.0,Alto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,34,1792.0
3,444926,7038,2022-08-05,512-3913163-0,20.0,MinimalUsage,65.26980,3.5263,14.0,Alto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19,555.0
4,450771,7030,2022-08-16,512-3957000-0,5.0,MediumUsage,209.26984,3.2222,17.0,Medio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,27,1759.0


# Preprocesamiento de Datos

In [11]:
# Encode categorical variables
label_encoders = {}
for col in ['BUSINESS_SEGMENT', 'CONCENTRATION', 'NSE', 'UNIQUE_SEGMENT', 'CHANNEL']:
    le = LabelEncoder()
    df_merged[col] = le.fit_transform(df_merged[col])
    label_encoders[col] = le

In [12]:
# Standardize numeric features
scaler = StandardScaler()
df_merged[['RECENCIA', 'FRECUENCIA', 'TOTAL_ITEMS']] = scaler.fit_transform(df_merged[['RECENCIA', 'FRECUENCIA', 'TOTAL_ITEMS']])

In [13]:
df_merged.isna().sum()

ACCOUNT_ID               0
SKU_ID                   0
INVOICE_DATE             0
ORDER_ID                 0
ITEMS_PHYS_CASES         0
BUSINESS_SEGMENT         0
TOTAL_VOLUME          2032
DIFF_SKU_BY_ORDER     2032
TOTAL_DIFF_SKU        2032
CONCENTRATION            0
NSE                      0
UNIQUE_SEGMENT           0
CHANNEL                  0
MES                      0
DIA                      0
DAY_OF_WEEK              0
LAG_1                 4535
LAG_2                 9035
LAG_3                13471
LAG_4                17833
LAG_5                22141
LAG_6                26380
LAG_7                30546
LAG_8                34633
LAG_9                38647
LAG_10               42590
LAG_11               46472
LAG_12               50273
LAG_13               54000
LAG_14               57655
MA_3                  9035
MA_6                 22141
MA_9                 34633
RECENCIA                 0
FRECUENCIA               0
TOTAL_ITEMS              0
dtype: int64

In [14]:
df_merged.dropna(inplace=True)

In [15]:
# Separate dataset into Train(June and July) and  Test(August)
train_data = df_merged[(df_merged['MES'].isin([6, 7]))]
test_data = df_merged[df_merged['MES'] == 8]

In [16]:
print(train_data.shape)
print(test_data.shape)

(140789, 36)
(81135, 36)


In [17]:
train_data.columns

Index(['ACCOUNT_ID', 'SKU_ID', 'INVOICE_DATE', 'ORDER_ID', 'ITEMS_PHYS_CASES',
       'BUSINESS_SEGMENT', 'TOTAL_VOLUME', 'DIFF_SKU_BY_ORDER',
       'TOTAL_DIFF_SKU', 'CONCENTRATION', 'NSE', 'UNIQUE_SEGMENT', 'CHANNEL',
       'MES', 'DIA', 'DAY_OF_WEEK', 'LAG_1', 'LAG_2', 'LAG_3', 'LAG_4',
       'LAG_5', 'LAG_6', 'LAG_7', 'LAG_8', 'LAG_9', 'LAG_10', 'LAG_11',
       'LAG_12', 'LAG_13', 'LAG_14', 'MA_3', 'MA_6', 'MA_9', 'RECENCIA',
       'FRECUENCIA', 'TOTAL_ITEMS'],
      dtype='object')

In [18]:
print(train_data['ACCOUNT_ID'].nunique())
print(test_data['ACCOUNT_ID'].nunique())

3462
3357


In [19]:
print(train_data['SKU_ID'].nunique())
print(test_data['SKU_ID'].nunique())

489
429


In [20]:
# Obtener los clientes del dataset de entrenamiento y prueba
train_clients = train_data['ACCOUNT_ID'].unique()
test_clients = test_data['ACCOUNT_ID'].unique()

# Obtener la intersección de ambos conjuntos de clientes
clientes_ambos_meses = list(set(train_clients).intersection(set(test_clients)))

# Filtrar los DataFrames para considerar solo esos clientes
train_filtered = train_data[train_data['ACCOUNT_ID'].isin(clientes_ambos_meses)]
test_filtered = test_data[test_data['ACCOUNT_ID'].isin(clientes_ambos_meses)]

print(f"Número de clientes en ambos meses: {len(clientes_ambos_meses)}")

Número de clientes en ambos meses: 3282


In [21]:
# Obtener listas únicas
clientes_unicos = transactions['ACCOUNT_ID'].unique()
productos_unicos = transactions['SKU_ID'].unique()

# Modelos de Recomendación

## Matriz Cliente-Producto

In [22]:
matriz_cliente_producto = df_merged.pivot_table(
    index='ACCOUNT_ID',  # Filas: Clientes
    columns='SKU_ID',  # Columnas: Productos
    values='ITEMS_PHYS_CASES',  # Valores: Cantidad comprada (ITEMS_PHYS_CASES)
    aggfunc='sum',  # Función de agregación: sumar los valores
    fill_value=0  # Rellenar con 0 para productos no comprados
)

In [23]:
# Mostrar la matriz
print(matriz_cliente_producto.shape)

(3538, 525)


In [24]:
matriz_cliente_producto.head()

SKU_ID,270,591,627,641,659,691,808,810,811,813,...,78078,78093,78094,78103,78104,78109,78121,78128,78129,78138
ACCOUNT_ID,,,,,,,,,,,,,,,,,,,,,
22247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22774,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Single Value Decomposition (SVD)

In [25]:
# Convertir la matriz cliente-producto a numpy array
matriz_cliente_producto_np = matriz_cliente_producto.values

# Aplicar la descomposición SVD
U, sigma, Vt = svds(matriz_cliente_producto_np, k=50)  # 'k' es el número de componentes latentes

# Convertir sigma a una matriz diagonal
sigma = np.diag(sigma)

In [26]:
# Reconstruir la matriz cliente-producto aproximada
matriz_reconstruida = np.dot(np.dot(U, sigma), Vt)

# Convertir la matriz reconstruida a un DataFrame
matriz_reconstruida_df = pd.DataFrame(matriz_reconstruida, index=matriz_cliente_producto.index, columns=matriz_cliente_producto.columns)

In [27]:
# Elegir un cliente (ejemplo: Cliente con ACCOUNT_ID = 1)
cliente_id = 420316

# Obtener las compras reales del cliente
productos_cliente_original = matriz_cliente_producto.loc[cliente_id]

# Obtener las predicciones de la matriz reconstruida
productos_predichos = matriz_reconstruida_df.loc[cliente_id]

# Combinar los productos comprados y no comprados, ordenados por score
recomendaciones = productos_predichos.sort_values(ascending=False)

# Mostrar las recomendaciones para el cliente
print(f"Recomendaciones para el cliente {cliente_id}:")
print(recomendaciones.head(10))

Recomendaciones para el cliente 420316:
SKU_ID
7038     1254.243437
19088     809.959639
24880     537.149513
7634      289.214281
19341     198.646240
26567     177.563504
7026      176.775727
19019      84.902044
22329      81.247379
2218       65.595830
Name: 420316, dtype: float64


# K-Nearest Neighbors (KNN)

In [28]:
# Crear el modelo KNN para clientes (usando la matriz U, que representa el espacio latente de los clientes)
knn_clientes = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_clientes.fit(U)

# Elegir un cliente para encontrar clientes similares (por ejemplo, el cliente con índice 0)
cliente_idx = 0

# Encontrar los clientes más similares en el espacio latente
distancias, indices = knn_clientes.kneighbors([U[cliente_idx]], n_neighbors=5)

# Mostrar los índices de los clientes más similares
print(f"Clientes similares al cliente {cliente_idx}: {indices.flatten()}")

Clientes similares al cliente 0: [   0 1610  547  278 1884]


In [29]:
# Transponer Vt para que las columnas representen los productos en el espacio latente
Vt_transpuesta = Vt.T

# Crear el modelo KNN para productos (usando la matriz Vt transpuesta)
knn_productos = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_productos.fit(Vt_transpuesta)

# Elegir un producto para encontrar productos similares (por ejemplo, el producto con índice 0)
producto_idx = 0

# Encontrar los productos más similares en el espacio latente
distancias, indices = knn_productos.kneighbors([Vt_transpuesta[producto_idx]], n_neighbors=5)

# Mostrar los índices de los productos más similares
print(f"Productos similares al producto {producto_idx}: {indices.flatten()}")

Productos similares al producto 0: [  0 516  36 311 352]


In [30]:
# Crear el modelo KNN basado en clientes
knn_model = NearestNeighbors(n_neighbors=5, metric='cosine')  # Puedes cambiar 'cosine' a 'euclidean'
knn_model.fit(matriz_cliente_producto)

NearestNeighbors(metric='cosine')

In [31]:
# Elegir un cliente para hacer recomendaciones (ejemplo: Cliente con ACCOUNT_ID = 1)
cliente_id = 420316

# Obtener el índice del cliente en la matriz
cliente_idx = matriz_cliente_producto.index.get_loc(cliente_id)

# Encontrar los clientes más similares
distancias, indices = knn_model.kneighbors([matriz_cliente_producto.iloc[cliente_idx]], n_neighbors=10)

# Mostrar los índices de los clientes más similares
print("Clientes similares:", matriz_cliente_producto.index[indices.flatten()])

Clientes similares: Index([420316, 423286, 452223, 411781, 311274, 417436, 448237, 388348, 456431,
       450959],
      dtype='int64', name='ACCOUNT_ID')


In [32]:
# Obtener los clientes similares (exceptuando el cliente original)
clientes_similares = matriz_cliente_producto.index[indices.flatten()][1:]  # Excluir el cliente original

# Productos que el cliente original ha comprado
productos_cliente_original = matriz_cliente_producto.loc[cliente_id]

# Productos comprados por los clientes similares
productos_similares = matriz_cliente_producto.loc[clientes_similares].mean(axis=0)

# Filtrar los productos que el cliente original ya ha comprado (productos_cliente_original > 0)
productos_comprados = productos_cliente_original[productos_cliente_original > 0]

# Filtrar los productos que el cliente similar ha comprado pero el cliente original no
productos_solo_similares = productos_similares[productos_cliente_original == 0]

# Combinar ambos: productos comprados por el cliente original y productos comprados por los similares
# Concatenamos las series
productos_totales = pd.concat([productos_comprados, productos_solo_similares])

# Ordenar todos los productos por su score en orden descendente
productos_ordenados = productos_totales.sort_values(ascending=False)

# Mostrar los productos ordenados por score
print("Productos comprados por el cliente y productos de clientes similares ordenados por score:")
print(productos_ordenados.head(10))

Productos comprados por el cliente y productos de clientes similares ordenados por score:
SKU_ID
7038     1254.0
19088     810.0
24880     541.0
7634      294.0
19341     200.0
7026      175.0
26567     175.0
22329      90.0
19019      85.0
2218       66.0
dtype: float64


## Popularidad por cantidad

In [33]:
def productos_mas_comprados(account_id, data, top_n):
    """
    Función para devolver los productos más comprados por un cliente específico.
    
    Parámetros:
    - account_id: ID del cliente para el cual se desea obtener los productos más comprados.
    - august_data: DataFrame que contiene los datos de compras, incluyendo ACCOUNT_ID, SKU_ID, ITEMS_PHYS_CASES.
    - top_n: Número de productos más comprados a devolver. Default es 5.
    
    Output:
    - DataFrame con 'ACCOUNT_ID', 'SKU_ID' y 'ITEMS_PHYS_CASES' sumados.
    """
    # Filtrar el conjunto de datos por el ACCOUNT_ID específico
    account_data = data[data['ACCOUNT_ID'] == account_id]
    
    # Agrupar por SKU_ID y sumar la cantidad de ITEMS_PHYS_CASES
    productos_comprados = account_data.groupby(['ACCOUNT_ID', 'SKU_ID'], as_index=False)['ITEMS_PHYS_CASES'].sum()
    
    # Ordenar por la cantidad de ITEMS_PHYS_CASES en orden descendente
    productos_comprados = productos_comprados.sort_values(by='ITEMS_PHYS_CASES', ascending=False)
    
    # Devolver los 'top_n' productos más comprados
    return productos_comprados.head(top_n)

In [34]:
# Ejemplo de uso
account_id = 420316  # ID del cliente
productos_top = productos_mas_comprados(account_id, df_merged, 10)
print(productos_top)

    ACCOUNT_ID  SKU_ID  ITEMS_PHYS_CASES
5       420316    7038            1254.0
17      420316   19088             810.0
30      420316   24880             541.0
6       420316    7634             294.0
19      420316   19341             200.0
3       420316    7026             175.0
35      420316   26567             175.0
24      420316   22329              90.0
16      420316   19019              85.0
1       420316    2218              66.0


In [35]:
# Ejemplo de uso
account_id = 420316  # ID del cliente
productos_top = productos_mas_comprados(account_id, train_filtered, 10)
print(productos_top)

    ACCOUNT_ID  SKU_ID  ITEMS_PHYS_CASES
5       420316    7038             930.0
12      420316   19088             600.0
21      420316   24880             248.0
14      420316   19341             150.0
26      420316   26567             105.0
11      420316   19019              85.0
6       420316    7634              84.0
27      420316   27086              45.0
16      420316   21810              33.0
28      420316   77682              30.0


In [36]:
# Ejemplo de uso
account_id = 420316  # ID del cliente
productos_top = productos_mas_comprados(account_id, test_filtered, 10)
print(productos_top)

    ACCOUNT_ID  SKU_ID  ITEMS_PHYS_CASES
4       420316    7038             324.0
22      420316   24880             293.0
5       420316    7634             210.0
13      420316   19088             210.0
3       420316    7026             150.0
18      420316   22329              90.0
24      420316   26567              70.0
14      420316   19341              50.0
1       420316    2218              41.0
15      420316   20433              25.0


# Modelo Supervisado : LightGBM

In [37]:
from lightfm import LightFM
from lightfm.data import Dataset

c:\Users\barra\anaconda3\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [38]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Definir características y variable objetivo
X = df_merged[['RECENCIA', 'FRECUENCIA', 'TOTAL_ITEMS', 'TOTAL_VOLUME', 'MES', 'DIA', 'DAY_OF_WEEK',
                          'LAG_1', 'LAG_2', 'LAG_3', 'LAG_4', 'LAG_5', 'LAG_6', 'MA_3', 'MA_6', 'MA_9',
                          'UNIQUE_SEGMENT', 'CONCENTRATION']]

y = df_merged['ITEMS_PHYS_CASES']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Crear el dataset para LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Configurar los parámetros del modelo
params = {
    'objective': 'regression',  # Para predecir valores continuos (volumen de compra)
    'metric': 'rmse',  # Error cuadrático medio como métrica
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Entrenar el modelo
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)

# Realizar predicciones
y_pred = model.predict(X_test)

# Calcular el RMSE (error cuadrático medio)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE del modelo: {rmse}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002789 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2217
[LightGBM] [Info] Number of data points in the train set: 177768, number of used features: 18
[LightGBM] [Info] Start training from score 3.706563
RMSE del modelo: 4.608284569278488


c:\Users\barra\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [39]:
def recomendar_productos(account_id, modelo, X_train_columns, df_completo):
    # Verificar si el ACCOUNT_ID existe en el DataFrame
    if account_id not in df_completo['ACCOUNT_ID'].unique():
        raise ValueError(f"El ACCOUNT_ID {account_id} no existe en el DataFrame")

    # Filtrar solo las filas correspondientes al ACCOUNT_ID
    productos_cliente = df_completo[df_completo['ACCOUNT_ID'] == account_id]

    # Seleccionar las características de estos productos usando las mismas columnas que en el entrenamiento
    X_cliente = productos_cliente[X_train_columns]

    # Hacer predicciones de score para estos productos
    score_pred = modelo.predict(X_cliente)

    # Crear el DataFrame con el ACCOUNT_ID, SKU_ID, y el SCORE predicho
    recomendaciones = pd.DataFrame({
        'ACCOUNT_ID': productos_cliente['ACCOUNT_ID'],
        'SKU_ID': productos_cliente['SKU_ID'],
        'SCORE': score_pred
    })

    # Agrupar por SKU_ID y calcular la media del SCORE para productos únicos
    recomendaciones_agrupadas = recomendaciones.groupby('SKU_ID').agg({
        'ACCOUNT_ID': 'first',  # Mantenemos el mismo ACCOUNT_ID
        'SCORE': 'sum'  # Calculamos la suma de los SCORES
    }).reset_index()

    # Ordenar por SCORE de mayor a menor
    recomendaciones_agrupadas = recomendaciones_agrupadas.sort_values(by='SCORE', ascending=False)

    return recomendaciones_agrupadas

In [40]:
# Ejemplo de uso: Recomendaciones para el cliente con ACCOUNT_ID 456111
account_id = 420316

# Asegúrate de que el ACCOUNT_ID existe en los datos
print(f"El ACCOUNT_ID {account_id} está presente:", account_id in train_filtered['ACCOUNT_ID'].unique())

# Generar recomendaciones
recomendaciones_cliente = recomendar_productos(account_id, model, X_train.columns, df_merged)

# Mostrar las primeras recomendaciones
print(recomendaciones_cliente[['ACCOUNT_ID', 'SKU_ID', 'SCORE']].head(10))

El ACCOUNT_ID 420316 está presente: True
    ACCOUNT_ID  SKU_ID        SCORE
5       420316    7038  1153.261365
17      420316   19088   742.516672
30      420316   24880   502.898557
6       420316    7634   277.700118
19      420316   19341   187.137281
35      420316   26567   176.106939
3       420316    7026   155.253323
1       420316    2218   119.244729
21      420316   21810    98.013926
24      420316   22329    80.547092


## Integración LightGBM con KNN considerando clientes y productos similares

In [41]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Seleccionar las características adicionales para KNN
caracteristicas_clientes = df_merged[['ACCOUNT_ID', 'RECENCIA', 'FRECUENCIA', 'TOTAL_ITEMS', 'TOTAL_VOLUME', 'MES', 'DIA', 'DAY_OF_WEEK',
                          'LAG_1', 'LAG_2', 'LAG_3', 'LAG_4', 'LAG_5', 'LAG_6', 'MA_3', 'MA_6', 'MA_9',
                          'UNIQUE_SEGMENT', 'CONCENTRATION']]

# Normalizar las características
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
caracteristicas_clientes_scaled = scaler.fit_transform(caracteristicas_clientes.drop(columns='ACCOUNT_ID'))

# Entrenar el modelo KNN
knn_clientes = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_clientes.fit(caracteristicas_clientes_scaled)

# Encontrar clientes similares
cliente_id = 456111  # Cliente para quien queremos encontrar similares
index_cliente = caracteristicas_clientes[caracteristicas_clientes['ACCOUNT_ID'] == cliente_id].index[0]
distancias, indices_similares = knn_clientes.kneighbors([caracteristicas_clientes_scaled[index_cliente]])

# Obtener los clientes similares
clientes_similares = caracteristicas_clientes.iloc[indices_similares[0]]['ACCOUNT_ID']
print("Clientes similares a:", cliente_id)
print(clientes_similares)

Clientes similares a: 456111
984       456111
486       456111
126449    456111
143127    456111
117047    456111
Name: ACCOUNT_ID, dtype: int64


In [42]:
# KNN para productos similares usando la matriz cliente-producto
knn_productos = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_productos.fit(matriz_cliente_producto.T)  # Transpuesta para buscar productos similares

# Encontrar productos similares a un SKU
sku_id = 7038  # ID del producto para encontrar productos similares
distancias_productos, indices_productos_similares = knn_productos.kneighbors([matriz_cliente_producto[sku_id]])

# Obtener los productos similares
productos_similares = matriz_cliente_producto.columns[indices_productos_similares[0]]
print("Productos similares a SKU_ID", sku_id)
print(productos_similares)

Productos similares a SKU_ID 7038
Index([7038, 26567, 24880, 19341, 19019], dtype='int64', name='SKU_ID')


In [43]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Seleccionar características para encontrar clientes similares
caracteristicas_clientes = df_merged[['RECENCIA', 'FRECUENCIA', 'TOTAL_ITEMS', 'ACCOUNT_ID']]

# Normalizar las características
scaler = StandardScaler()
caracteristicas_clientes_scaled = scaler.fit_transform(caracteristicas_clientes.drop(columns='ACCOUNT_ID'))

# Entrenar el modelo KNN
knn_clientes = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_clientes.fit(caracteristicas_clientes_scaled)

# Encontrar los vecinos más cercanos para cada cliente
distancias, indices_similares = knn_clientes.kneighbors(caracteristicas_clientes_scaled)

# Crear una nueva característica con el promedio de los vecinos más cercanos para cada cliente
similitud_clientes = distancias.mean(axis=1)
df_merged['SIMILITUD_CLIENTES'] = similitud_clientes

In [44]:
# Revisar las primeras filas para confirmar el merge
print(df_merged.head())

     ACCOUNT_ID  SKU_ID INVOICE_DATE       ORDER_ID  ITEMS_PHYS_CASES  \
248      174250   24012   2022-06-03  512-3648097-0              10.0   
250      456111   26567   2022-08-04  512-3910904-0             100.0   
262      456111   24880   2022-08-10  512-3917941-0              50.0   
267      174250   18565   2022-07-12  512-3799614-0              10.0   
268      174250    1427   2022-07-06  512-3773773-0              10.0   

     BUSINESS_SEGMENT  TOTAL_VOLUME  DIFF_SKU_BY_ORDER  TOTAL_DIFF_SKU  \
248                 2     616.67710             8.0500           100.0   
250                 0    4274.44416             2.3436            32.0   
262                 0    4274.44416             2.3436            32.0   
267                 2     616.67710             8.0500           100.0   
268                 2     616.67710             8.0500           100.0   

     CONCENTRATION  ...  LAG_12  LAG_13  LAG_14        MA_3        MA_6  \
248              0  ...    15.0    40.0  

In [45]:
# Crear la matriz cliente-producto
matriz_cliente_producto = df_merged.pivot_table(index='ACCOUNT_ID', columns='SKU_ID', values='ITEMS_PHYS_CASES', fill_value=0)

# Entrenar el modelo KNN para productos
knn_productos = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_productos.fit(matriz_cliente_producto.T)

# Encontrar productos similares para cada SKU_ID
distancias_productos, indices_productos_similares = knn_productos.kneighbors(matriz_cliente_producto.T)

# Crear un DataFrame para las similitudes de productos
similitud_productos_df = pd.DataFrame({
    'SKU_ID': matriz_cliente_producto.columns,
    'SIMILITUD_PRODUCTOS': distancias_productos.mean(axis=1)  # Promedio de las distancias de los productos más cercanos
})

# Hacer un merge con el df_merged original basado en SKU_ID
df_merged = pd.merge(df_merged, similitud_productos_df, on='SKU_ID', how='left')

# Mostrar los primeros resultados
print(df_merged.head())

   ACCOUNT_ID  SKU_ID INVOICE_DATE       ORDER_ID  ITEMS_PHYS_CASES  \
0      174250   24012   2022-06-03  512-3648097-0              10.0   
1      456111   26567   2022-08-04  512-3910904-0             100.0   
2      456111   24880   2022-08-10  512-3917941-0              50.0   
3      174250   18565   2022-07-12  512-3799614-0              10.0   
4      174250    1427   2022-07-06  512-3773773-0              10.0   

   BUSINESS_SEGMENT  TOTAL_VOLUME  DIFF_SKU_BY_ORDER  TOTAL_DIFF_SKU  \
0                 2     616.67710             8.0500           100.0   
1                 0    4274.44416             2.3436            32.0   
2                 0    4274.44416             2.3436            32.0   
3                 2     616.67710             8.0500           100.0   
4                 2     616.67710             8.0500           100.0   

   CONCENTRATION  ...  LAG_13  LAG_14        MA_3        MA_6        MA_9  \
0              0  ...    40.0    70.0   10.000000   10.000000  

In [56]:
import lightgbm as lgb

# Seleccionar las características para LightGBM, incluyendo las similitudes calculadas
X = df_merged[['RECENCIA', 'FRECUENCIA', 'TOTAL_ITEMS', 'TOTAL_VOLUME', 'MES', 'DIA', 'DAY_OF_WEEK',
                          'LAG_1', 'LAG_2', 'LAG_3', 'LAG_4', 'LAG_5', 'LAG_6', 'MA_3', 'MA_6', 'MA_9',
                          'UNIQUE_SEGMENT', 'CONCENTRATION', 'SIMILITUD_CLIENTES', 'SIMILITUD_PRODUCTOS']]

# Variable objetivo
y = df_merged['ITEMS_PHYS_CASES']

# Dividir los datos en conjunto de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el dataset para LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Configurar los parámetros de LightGBM
params = {
    'objective': 'regression',  # Si quieres predecir el volumen de compra
    'metric': 'rmse',  # Para regresión
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Entrenar el modelo
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE del modelo LightGBM con KNN: {rmse}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011106 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2588
[LightGBM] [Info] Number of data points in the train set: 177768, number of used features: 20
[LightGBM] [Info] Start training from score 3.706563
RMSE del modelo LightGBM con KNN: 4.732380919115376


c:\Users\barra\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [57]:
def recomendar_productos(account_id, modelo_knn, modelo_lgbm, df_completo, X_train_columns, top_n=10):
    """
    Función para recomendar productos basada en el modelo KNN (productos similares) y LightGBM (predicciones).
    Incluye tanto productos que el cliente ya ha comprado como productos similares.
    Agrupa por SKU_ID y calcula el promedio del score predicho para recomendar productos únicos.
    """
    # 1. Filtrar productos que el cliente ya ha comprado
    productos_cliente = df_completo[df_completo['ACCOUNT_ID'] == account_id]['SKU_ID'].unique()
    
    print(f"Productos comprados por el cliente {account_id}: {productos_cliente}")
    
    # Filtrar productos disponibles que el cliente no ha comprado
    productos_no_comprados = df_completo[~df_completo['SKU_ID'].isin(productos_cliente)]['SKU_ID'].unique()

    # 2. Encontrar productos similares usando el modelo KNN para productos no comprados
    productos_similares = []
    for producto in productos_cliente:
        distancias, indices = modelo_knn.kneighbors([matriz_cliente_producto[producto]])
        productos_similares.extend(matriz_cliente_producto.columns[indices[0]])  # Agregar productos similares

    # Filtrar los productos similares que el cliente aún no ha comprado
    productos_similares = [p for p in productos_similares if p in productos_no_comprados]
    
    print(f"Productos similares y no comprados por {account_id}: {productos_similares}")

    # 3. Combinar productos ya comprados y productos similares no comprados
    productos_recomendados = list(productos_cliente) + productos_similares

    # Si no hay productos recomendados, devolver un mensaje
    if len(productos_recomendados) == 0:
        return pd.DataFrame({"Mensaje": ["No hay productos disponibles para recomendar."]})

    # 4. Preparar los datos para LightGBM usando las características de los productos recomendados
    recomendaciones_df = df_completo[df_completo['SKU_ID'].isin(productos_recomendados)].copy()
    recomendaciones_df = recomendaciones_df[recomendaciones_df['ACCOUNT_ID'] == account_id]

    print(f"Filas en recomendaciones_df después del filtrado: {len(recomendaciones_df)}")

    # Verificar que las columnas de X_train_columns existan en recomendaciones_df
    columnas_faltantes = [col for col in X_train_columns if col not in recomendaciones_df.columns]
    if columnas_faltantes:
        raise ValueError(f"Las siguientes columnas faltan en el DataFrame de recomendaciones: {columnas_faltantes}")

    # Seleccionar las características usadas en LightGBM
    X_recomendaciones = recomendaciones_df[X_train_columns]

    # Verificar que X_recomendaciones no esté vacío
    if X_recomendaciones.empty:
        raise ValueError(f"No hay datos para hacer recomendaciones para el ACCOUNT_ID {account_id}")

    # 5. Predecir el score de compra usando LightGBM
    score_pred = modelo_lgbm.predict(X_recomendaciones)

    # 6. Crear un DataFrame con las predicciones
    recomendaciones_finales = pd.DataFrame({
        'ACCOUNT_ID': recomendaciones_df['ACCOUNT_ID'],
        'SKU_ID': recomendaciones_df['SKU_ID'],
        'PREDICTED_SCORE': score_pred
    })

    # 7. Agrupar por SKU_ID y calcular el promedio del PREDICTED_SCORE
    recomendaciones_agrupadas = recomendaciones_finales.groupby('SKU_ID').agg({
        'ACCOUNT_ID': 'first',  # Mantener el ACCOUNT_ID
        'PREDICTED_SCORE': 'sum'  # Promedio del score
    }).reset_index()

    # 8. Ordenar las recomendaciones por el score promedio
    recomendaciones_agrupadas = recomendaciones_agrupadas.sort_values(by='PREDICTED_SCORE', ascending=False)

    # 9. Devolver los 'top_n' productos recomendados
    return recomendaciones_agrupadas.head(top_n)

In [58]:
# Ejemplo de uso:
# - modelo_knn: Modelo KNN entrenado para productos
# - modelo_lgbm: Modelo LightGBM entrenado para predecir compras
account_id = 420316
recomendaciones = recomendar_productos(account_id, knn_productos, model, df_merged, X_train.columns)

# Mostrar las primeras recomendaciones
print(recomendaciones[['ACCOUNT_ID', 'SKU_ID', 'PREDICTED_SCORE']])

Productos comprados por el cliente 420316: [16668 24880  7026  7038 19341  2218 21810 19088 19019 20433  7634 24598
 16667  7030 26567 24878 27086 17932 25686 25188  1416 22321 77934 22320
 22652 22329  7713  7651 16811 77682 25352 19253 26527 16936 16666  2439
 23112 16527 24075]
Productos similares y no comprados por 420316: [24378, 21863, 15238, 77832, 7476, 77812, 19399, 17991, 25120, 77680, 77861, 77812, 76504, 77680, 18377, 24931, 77782, 24968, 77832, 19254, 7484, 15094, 19587, 22390, 25190, 25184, 20996, 25346, 1418, 76477, 77812, 13950, 77833, 22328, 25135, 24642, 77684, 24011, 77677, 77935, 76369, 22328, 76463, 25184, 25689, 22607, 25346, 76483, 77812, 76386, 26427, 25490, 26995, 18746, 76476, 17933, 19254, 18361, 26707, 76411, 77835, 76447, 76342, 76504, 26427, 25490, 25689, 18377, 18354, 17993, 77861, 77680, 11522, 21677, 25190, 20996, 78078, 20471, 25770, 23287, 77964, 10557, 16529, 2428]
Filas en recomendaciones_df después del filtrado: 181
    ACCOUNT_ID  SKU_ID  PREDICTE